In [ ]:
import pandas as pd
import numpy as np

In [ ]:
twitterDataFrame=pd.read_csv('/content/twitter_training.csv')

In [ ]:
twitterDataFrame.head(5)

,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [ ]:
twitterFiltredDataFrame=twitterDataFrame.iloc[:,1:]

In [ ]:
twitterFiltredDataFrame

,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,Borderlands,Positive,I am coming to the borders and I will kill you...
1,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,Borderlands,Positive,im coming on borderlands and i will murder you...
3,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,Borderlands,Positive,im getting into borderlands and i can murder y...
...,...,...,...
74676,Nvidia,Positive,Just realized that the Windows partition of my...
74677,Nvidia,Positive,Just realized that my Mac window partition is ...
74678,Nvidia,Positive,Just realized the windows partition of my Mac ...
74679,Nvidia,Positive,Just realized between the windows partition of...


In [ ]:
twitterFiltredDataFrame.columns=["candidate","sentiment","text"]

In [ ]:
twitterFiltredDataFrame['text'].dtype

dtype('O')

In [ ]:
def clean_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', str(text))  # Remove punctuation & numbers
    text = text.lower()  # Convert to lowercase
    text = text.strip()  # Remove leading/trailing spaces
    return text

In [ ]:
import re
twitterFiltredDataFrame['text'].apply(clean_text)

,text
0,i am coming to the borders and i will kill you...
1,im getting on borderlands and i will kill you all
2,im coming on borderlands and i will murder you...
3,im getting on borderlands and i will murder y...
4,im getting into borderlands and i can murder y...
...,...
74676,just realized that the windows partition of my...
74677,just realized that my mac window partition is ...
74678,just realized the windows partition of my mac ...
74679,just realized between the windows partition of...


In [ ]:
twitterFiltredDataFrame.isnull().sum()

,0
candidate,0
sentiment,0
text,686


In [ ]:
twitterFiltredDataFrame.dropna(inplace=True)

In [ ]:
twitterFiltredDataFrame.isnull().sum()

,0
candidate,0
sentiment,0
text,0


In [ ]:
twitterFiltredDataFrame['sentiment'].value_counts()

,count
sentiment,
Negative,22358
Positive,20654
Neutral,18108
Irrelevant,12875


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocabSize = 20000  # Choose vocabulary size
outofVocabularyToken = "<OOV>"  # Token for Out-Of-Vocabulary words

tokenizer = Tokenizer(num_words=vocabSize, oov_token=outofVocabularyToken)
tokenizer.fit_on_texts(twitterFiltredDataFrame['text'])  # Learn the word indices


In [ ]:
word_index = tokenizer.word_index

In [ ]:
# Step 2: Convert text to sequences
sequences = tokenizer.texts_to_sequences(twitterFiltredDataFrame['text'])

In [ ]:
len(sequences)

73995

In [ ]:
max_len = max(sequences, key=lambda x: len(x))  # Finds the longest sequence
max_length = len(max_len)

In [ ]:
max_length

166

In [ ]:
# Step 3: Padding sequences

padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')

In [ ]:
len(tokenizer.word_index) + 1

33785

In [ ]:
padded_sequences.shape

(73995, 166)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SpatialDropout1D, LSTM, Dense,Bidirectional
from tensorflow.keras.optimizers import Adam


In [ ]:
from tensorflow.keras.regularizers import l2

model = Sequential()
model.add(Embedding(input_dim=33785, output_dim=256, input_length=166))  # Explicit input_length
model.add(SpatialDropout1D(0.2))
model.add(Bidirectional(LSTM(166, return_sequences=True)))  # Keeps sequence for next LSTM
model.add(Bidirectional(LSTM(166, kernel_regularizer=l2(0.01))))  # Final LSTM outputs last state
model.add(Dense(4, activation='softmax', kernel_regularizer=l2(0.01)))  # 4 output classes
model.build(input_shape=(None, 166))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
from tensorflow.keras.optimizers import AdamW

modeloptimizer = AdamW(learning_rate=0.00001, weight_decay=1e-4)  # Weight decay for regularization
model.compile(optimizer=modeloptimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 166, 256)            │       8,648,960 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ spatial_dropout1d (SpatialDropout1D) │ (None, 166, 256)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 166, 332)            │         561,744 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 332)                 │         662,672 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 4)                   │           1,332 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 9,874,708 (37.67 MB)

 Trainable params: 9,874,708 (37.67 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
twitterFiltredDataFrame['sentiment'].shape

(73995,)

In [ ]:
padded_sequences.shape

(73995, 166)

In [ ]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(sampling_strategy='auto', random_state=42)
padded_sequences_resampled, y_resampled = ros.fit_resample(padded_sequences, twitterFiltredDataFrame['sentiment'])

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y = encoder.fit_transform(y_resampled)
y = pd.get_dummies(y).values

In [ ]:
padded_sequences_resampled.shape

(89432, 166)

In [ ]:
y.shape

(89432, 4)

In [ ]:
import numpy as np

history = model.fit(padded_sequences_resampled,y, epochs=25, batch_size=64, validation_split=0.2)

Epoch 1/25
1118/1118 ━━━━━━━━━━━━━━━━━━━━ 80s 71ms/step - accuracy: 0.7557 - loss: 0.7876 - val_accuracy: 0.6365 - val_loss: 1.0682
Epoch 2/25
1118/1118 ━━━━━━━━━━━━━━━━━━━━ 79s 69ms/step - accuracy: 0.7665 - loss: 0.7597 - val_accuracy: 0.6505 - val_loss: 1.0631
Epoch 3/25
1118/1118 ━━━━━━━━━━━━━━━━━━━━ 81s 72ms/step - accuracy: 0.7744 - loss: 0.7414 - val_accuracy: 0.6437 - val_loss: 1.1013
Epoch 4/25
1118/1118 ━━━━━━━━━━━━━━━━━━━━ 82s 72ms/step - accuracy: 0.7799 - loss: 0.7165 - val_accuracy: 0.6794 - val_loss: 1.0073
Epoch 5/25
1118/1118 ━━━━━━━━━━━━━━━━━━━━ 78s 69ms/step - accuracy: 0.7880 - loss: 0.6975 - val_accuracy: 0.6763 - val_loss: 1.0173
Epoch 6/25
1118/1118 ━━━━━━━━━━━━━━━━━━━━ 81s 72ms/step - accuracy: 0.7937 - loss: 0.6798 - val_accuracy: 0.6793 - val_loss: 1.0041
Epoch 7/25
1118/1118 ━━━━━━━━━━━━━━━━━━━━ 78s 69ms/step - accuracy: 0.7983 - loss: 0.6658 - val_accuracy: 0.6854 - val_loss: 0.9826
Epoch 8/25
1118/1118 ━━━━━━━━━━━━━━━━━━━━ 77s 69ms/step - accuracy: 0.8070 -

In [ ]:
sentence = "worst movie"

In [ ]:
# Preprocess the new reviews
new_reviews_cleaned = clean_text(sentence)
new_reviews_seq = tokenizer.texts_to_sequences(new_reviews_cleaned)
new_reviews_padded = pad_sequences(new_reviews_seq, maxlen=166)
predictions = model.predict(new_reviews_padded)
# Convert predictions to readable output
predicted_classes = np.argmax(predictions, axis=1)
predicted_sentiments = encoder.inverse_transform([predicted_classes[0]])
print(predicted_sentiments)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
['Negative']
